## 使用SageMaker的BYOS(Bring Your Oen Script)方式部署大语言模型

利用SageMaker部署大语言模型的原理如下：

<img src="imgs/sagemaker_deploy_model.jpg" style="width: 850px;"></img>

SageMaker提供了几种不同的Container来部署模型，这里我们就使用默认提供的容器来进行部署。

### 可部署的模型

这里提供了多个模型，以及相应的script用于部署，提供的模型有：
 * ChatGLM
 * ChatGLM2
 * Baichuan
 * Baichuan2
 * cn-alpaca
 * Chatyuan-v2
 * CPM-Bee-5B

所需的脚本在相应的`code-*`文件夹里。


### 部署

准备：
1. 升级boto3, sagemaker python sdk  
2. 准备inference.py, requirements.txt

In [1]:
# 如果需要，更新sagemaker和 aws python sdk boto3
# !pip install --upgrade boto3
# !pip install --upgrade sagemaker

In [1]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

In [2]:
bucket

'sagemaker-us-east-1-568765279027'

接下来，我们使用Sagemaker进行模型部署。

所有 “code-” 开始的文件夹都是以BYOS的方式部署时的脚本文件。

In [25]:
# model_dir = "cn-alpaca2-7b"
# model_dir = "cpmbee"
model_dir = "baichuan2-13b-4bits"
# model_dir = "chatglm2"
source_dir = './code-' + model_dir
# model_code = "chatglm"
model_code = "baichuan2"

In [26]:
from sagemaker.pytorch.model import PyTorchModel

model_name = None
framework_version = '2.0.0'
py_version = 'py310'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

!touch dummy
!tar czvf model.tar.gz dummy
!rm -f dummy

model = PyTorchModel(
    name = model_dir + "-model",
    model_data = "s3://sagemaker-us-east-1-568765279027/mt_models_uploaded/baichuan-inc--Baichuan2-13B-Chat-4bits-tar/model.tar.gz",
    entry_point = 'inference.py',
    source_dir = source_dir,
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

dummy


endpoint_name = 'mt-'+model_dir+'-g4dn'然后部署该模型为 Sagemaker endpoint

In [27]:
instance_type = 'ml.g4dn.2xlarge'
# instance_type = 'ml.g5.2xlarge'

# endpoint_name = 'mt-'+model_dir+'-g5'
endpoint_name = 'mt-'+model_dir+'-g4dn'
endpoint_name

'mt-baichuan2-13b-4bits-g4dn'

In [29]:
# deploy as sagemaker endpoint
instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type,
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

--------------!

如果部署过程中出现错误，部署失败，通过下面的方式删除endpoint以及相应的模型。

In [32]:
from sagemaker import serializers, deserializers

del_predictor = sagemaker.Predictor(
    endpoint_name='mt-baichuan2-13b-4bits-g4dn', 
    sagemaker_session=sagemaker_session, 
    serializer=serializers.JSONSerializer(), 
    deserializer=deserializers.JSONDeserializer())
del_predictor.delete_model()
del_predictor.delete_endpoint()

如果重启了kernal，可以用下面的方式获取predictor

In [87]:
from sagemaker import serializers, deserializers

predictor = sagemaker.Predictor(
    endpoint_name="mt-chatglm2-g4dn", 
    sagemaker_session=sagemaker_session, 
    serializer=serializers.JSONSerializer(), 
    deserializer=deserializers.JSONDeserializer()
)

In [11]:
# from prompt_generator import generate_prompt as gen_prompt
import prompt_generator as pg
import importlib

importlib.reload(pg)

<module 'prompt_generator' from '/home/ec2-user/SageMaker/LLM_Workshop/prompt_generator.py'>

In [12]:
model_code

'chatglm'

In [33]:
# pg.generate_prompt("写一首关于交通信号灯的诗")

payload = pg.create_payload("信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案：", model_code)
predictor.predict(payload)

ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint mt-baichuan2-13b-4bits-g4dn of account 568765279027 not found.

In [ ]:
inputs= [
    pg.create_payload("写一首关于交通信号灯的诗", model_code),
    pg.create_payload("陨石为什么总能落在陨石坑里?", model_code),
    pg.create_payload("为什么爸妈结婚没叫我参加婚礼?", model_code)
]
response = predictor.predict(inputs[0])
print("\n\n问题: ", inputs[0]["inputs"], "\n回答:\n", response["outputs"])
response = predictor.predict(inputs[1])
print("\n\n问题: ", inputs[1]["inputs"], "\n回答:\n", response["outputs"])
response = predictor.predict(inputs[2])
print("\n\n问题: ", inputs[2]["inputs"], "\n回答:\n", response["outputs"])

### 通过Sagemaker Endpoint调用
我们已经将模型部署到了Sagemaker Endpoint上，我们就可以通过这个Endpoint名称，来调用模型进行推理，这样即使你停止了这个notebook，也能使用已经部署的模型。

In [8]:
# endpoint_name = "mt-chatglm2-6b-ft-g4dn"
# endpoint_name = "mt-chatglm2-g4dn-pytorch2"
endpoint_name = 'mt-baichuan2-13b-4bits-g4dn'
# endpoint_name = "mt-cpmbee-g4dn"

print(endpoint_name)
print(model_code)

mt-baichuan2-13b-4bits-g4dn
chatglm


In [9]:
import json
import boto3

client = boto3.client('runtime.sagemaker')

def query_endpoint(encoded_json):
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/json', Body=encoded_json)
    model_predictions = json.loads(response['Body'].read())
    generated_text = model_predictions["outputs"]
    return generated_text



In [13]:
%%time

payload = pg.create_payload("信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案：", model_code)

resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from primary with message "Your invocation timed out while waiting for a response from container primary. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/mt-baichuan2-13b-4bits-g4dn in account 568765279027 for more information.

In [48]:
%%timeit -r4
payload = pg.create_payload("天为什么是蓝色的？", model_code)
answer = query_endpoint(json.dumps(payload).encode('utf-8'))
print(answer)
print(len(answer))

天空之所以呈现蓝色,是由于光的散射现象造成的。当太阳光穿过大气层时,光被大气中的分子和小颗粒散射,这些颗粒包括氧气、氮气和水蒸气等。这些分子和颗粒吸收较短波长的光,如紫色和蓝色,而较长波长的光,如红色和橙色则被散射得更少。

由于蓝色光的波长比红色光短,因此它更容易被散射,而在大气层中被散射的蓝色光会向各个方向散射,包括朝着我们的眼睛,因此天空呈现出蓝色。在日出和日落时,太阳光穿过更长的大气层路径,较多的光被散射,因此天空呈现出更温暖的色调,如红色、橙色和黄色。
233
天空之所以呈现蓝色,是由于光的散射现象造成的。当太阳光穿过大气层时,光被大气中的分子和小颗粒散射,这些颗粒包括氧气、氮气和水蒸气等。这些分子和颗粒吸收较短波长的光,如紫色和蓝色,而较长波长的光,如红色和橙色则被散射得更少。

由于蓝色光的波长比红色光短,因此它更容易被散射,而在大气层中被散射的蓝色光会向各个方向散射,包括朝着我们的眼睛,因此天空呈现出蓝色。在日出和日落时,太阳光穿过更长的大气层路径,较多的光被散射,因此天空呈现出更温暖的色调,如红色、橙色和黄色。
233
天空之所以呈现蓝色,是由于光的散射现象造成的。当太阳光穿过大气层时,光被大气中的分子和小颗粒散射,这些颗粒包括氧气、氮气和水蒸气等。这些分子和颗粒吸收较短波长的光,如紫色和蓝色,而较长波长的光,如红色和橙色则被散射得更少。

由于蓝色光的波长比红色光短,因此它更容易被散射,而在大气层中被散射的蓝色光会向各个方向散射,包括朝着我们的眼睛,因此天空呈现出蓝色。在日出和日落时,太阳光穿过更长的大气层路径,较多的光被散射,因此天空呈现出更温暖的色调,如红色、橙色和黄色。
233
天空之所以呈现蓝色,是由于光的散射现象造成的。当太阳光穿过大气层时,光被大气中的分子和小颗粒散射,这些颗粒包括氧气、氮气和水蒸气等。这些分子和颗粒吸收较短波长的光,如紫色和蓝色,而较长波长的光,如红色和橙色则被散射得更少。

由于蓝色光的波长比红色光短,因此它更容易被散射,而在大气层中被散射的蓝色光会向各个方向散射,包括朝着我们的眼睛,因此天空呈现出蓝色。在日出和日落时,太阳光穿过更长的大气层路径,较多的光被散射,因此天空呈现出更温暖的色调,如红色、橙色和黄色。
233
天空之所以呈现蓝色,是由于光的散射现象造成的。当太阳光穿过大气层时,光被大气中的分子和小颗粒散射

In [96]:
%%time
payload = pg.create_payload("小明有200元，买书花了66元，买文具花了12元，小明剩下的钱比原来少了多少元？", model_code)
answer = query_endpoint(json.dumps(payload).encode('utf-8'))
print(answer)
print(len(answer))

首先，小明有200元钱，买书花了66元，买文具花了12元。那么，小明剩下的钱数是多少呢？我们可以用减法来计算：200-66-12=122元。

接下来，我们需要求出小明剩下的钱比原来少了多少元。我们可以用减法来计算：200-122=78元。

因此，小明剩下的钱比原来少了78元。
140
CPU times: user 32.1 ms, sys: 0 ns, total: 32.1 ms
Wall time: 5.82 s


In [97]:
%%time
payload = pg.create_payload("4加2等于几？", model_code)
resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

4加2等于6。
CPU times: user 4.83 ms, sys: 0 ns, total: 4.83 ms
Wall time: 450 ms


In [100]:
%%time

payload = pg.create_payload("USER：3加8等于几？\nASSISTANT：3加8等于11。 \nUSER：再乘以3呢？\nASSISTANT:", model_code)

resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

3加8等于11,再乘以3还是33。
CPU times: user 4.7 ms, sys: 0 ns, total: 4.7 ms
Wall time: 935 ms


### 删除 EndPoint

In [24]:
predictor.delete_model()
predictor.delete_endpoint()